In [ ]:
!pip install d2l==0.16.

In [2]:
import torchvision
from torch.utils import data
from torchvision import transforms
def load_data_fashion_mnist(batch_size, resize=None):
    """下载Fashion-MNIST数据集，然后将其加载到内存中"""
    trans = [transforms.ToTensor()]
    if resize:
        trans.insert(0, transforms.Resize(resize))
    trans = transforms.Compose(trans)
    mnist_train = torchvision.datasets.FashionMNIST(
        root="../data", train=True, transform=trans, download=True)
    mnist_test = torchvision.datasets.FashionMNIST(
        root="../data", train=False, transform=trans, download=True)
 # 注意：colab只支持2线程，所以这里的num_works为2
    return (data.DataLoader(mnist_train, batch_size, shuffle=True,
                            num_workers=2),
            data.DataLoader(mnist_test, batch_size, shuffle=False,
                            num_workers=2))




In [3]:
from google.colab import drive
drive.mount('/content/drive')

%cd /content/drive/My Drive
!ls
%cd d2l-dataset/

Mounted at /content/drive
/content/drive/My Drive
'Colab Notebooks'   d2l-dataset   d2l-zh   data   test.ipynb
/content/drive/My Drive/d2l-dataset


In [4]:
%matplotlib inline
import torch
from d2l import torch as d2l
from torch import nn

def cls_predictor(num_inputs ,num_anchors, num_classes):
  return nn.Conv2d(num_inputs,num_anchors*(num_classes +1),
                   kernel_size=2,padding=1)
  

In [6]:
def bbox_predictor(num_inputs, num_anchors):
  return nn.Conv2d(num_inputs,num_anchors*4,kernel_size=3,padding=1)

In [24]:
def flatten_pred(pred):
  return torch.flatten(pred.permute(0,2,3,1),start_dim=1)

def concat_pred(preds):
  return torch.concat([flatten_pred(x) for x in preds],dim=1);


In [22]:
def forward(x,blocks):
  return (blocks(x))

Y1 = forward(torch.zeros((2,8,20,20)),cls_predictor(8,5,10))
Y2 = forward(torch.zeros((2,16,10,10)),cls_predictor(16,3,10))
Y1.shape  ,Y2.shape

(torch.Size([2, 55, 21, 21]), torch.Size([2, 33, 11, 11]))

In [25]:
flatten_pred(Y1).shape,flatten_pred(Y2).shape,concat_pred([Y1,Y2]).shape

(torch.Size([2, 24255]), torch.Size([2, 3993]), torch.Size([2, 28248]))

In [36]:
def down_sample_blk(in_channels,out_channels):
  blk = []
  for _ in range(2):
    blk.append(nn.Conv2d(in_channels, out_channels,
                             kernel_size=3, padding=1))
    blk.append(nn.BatchNorm2d(out_channels))
    blk.append(nn.ReLU())
    in_channels = out_channels
  blk.append(nn.MaxPool2d(2))
  return nn.Sequential(*blk)

In [38]:
forward(torch.zeros((2,3,20,20)),down_sample_blk(3,10)).shape

torch.Size([2, 10, 10, 10])

In [44]:
def base_net():
  blk =[]
  num_filters = [3,16,32,64]
  for i in range(len(num_filters) - 1):
    blk.append(down_sample_blk(num_filters[i],num_filters[i+1]))
  
  return nn.Sequential(*blk)

In [45]:
forward(torch.zeros((2,3,256,256)),base_net()).shape

torch.Size([2, 64, 32, 32])

In [46]:
def get_blk(i):
  if i==0:
    blk = base_net()
  elif i==1:
    blk = down_sample_blk(64,128)
  elif i==4:
    blk = nn.AdaptiveAvgPool2d((1,1))
  else:
    blk = down_sample_blk(128,128)
  
  return blk

In [49]:
def blk_forward(x,blk,size,ratio,cls_predictor,bbox_predictor):
  Y = blk(x)
  anchors = d2l.miltibox_prior(Y,sizes = size,ratios=ratio)
  cls_preds = cls_predictor(Y)
  bbox_preds = bbox_predictor(Y)

  return (Y,anchors,cls_preds,bbox_preds)

In [50]:
sizes = [[0.2,0.272],[0.37,0.447],[0.54,0.619],[0.71,0.79],[0.88,0.961]]
ratios = [[1,2,0.5]]
num_anchors = len(sizes[0]) + len(ratios[0]) - 1

In [ ]:
class TinySSD(nn.Module):
  def __init__(self,num_classes,**kwargs):
    super(TinySSD,self).__init__(**kwargs)
    self.num_classes = num_classes
    idx_to_in_channels = [64,128,128,128,128]
    for i in range(len(idx_to_in_channels)):
      setattr(self,f"blk_{i}",get_blk(i))
      setattr(self,f"cls_{i}",cls_predictor(idx_to_in_channels[i],num_anchors,num_classes))
      setattr(self,f"bbox_{i}",cls_predictor(idx_to_in_channels[i],num_anchors,num_classes))
    
   def forward(self, X):
      anchors, cls_preds, bbox_preds = [None] * 5, [None] * 5, [None] * 5
      for i in range(5):
          # getattr(self,'blk_%d'%i)即访问self.blk_i
           X, anchors[i], cls_preds[i], bbox_preds[i] = blk_forward(
              X, getattr(self, f'blk_{i}'), sizes[i], ratios[i],
              getattr(self, f'cls_{i}'), getattr(self, f'bbox_{i}'))
      anchors = torch.cat(anchors, dim=1)
      cls_preds = concat_pred(cls_preds)
      cls_preds = cls_preds.reshape(
          cls_preds.shape[0], -1, self.num_classes + 1)
      bbox_preds = concat_pred(bbox_preds)
      return anchors, cls_preds, bbox_preds